# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login

Already logged in to 'https://app.truefoundry.com' as 'debajyoti-tfy' ('debajyoti@truefoundry.com')
Please use `sfy login --relogin` or `sfy.login(relogin=True)` to force relogin



In [ ]:
!git clone https://github.com/truefoundry/truefoundry-examples.git
%cd truefoundry-examples
!git checkout demo-add
%cd end-to-end-examples/diabetes-regression

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE="tfy-cluster-euwe1:demos"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

Please enter your API Key,( https://app.truefoundry.com/settings ):-··········


# ⚡ Training a ML Model using Jobs
In this section we will deploy a training job that trains a SVM model on iris dataset

In [ ]:
! mkdir -p /content/demo/regression/
%cd /content/demo/regression/

/content/demo/regression


In [ ]:
%%writefile train.py
import argparse

import matplotlib.pyplot as plt
import mlfoundry
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import PredictionErrorDisplay


def train(kernel: str):
    X, y = load_diabetes(as_frame=True, return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    regressor = SVR(kernel=kernel)
    model = TransformedTargetRegressor(
        regressor=regressor,
        transformer=QuantileTransformer(n_quantiles=100, output_distribution="normal"),
    ).fit(X_train, y_train)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    PredictionErrorDisplay.from_predictions(
        y_test,
        y_pred,
        kind="actual_vs_predicted",
        scatter_kwargs={"alpha": 0.5},
    )

    run = mlfoundry.get_client().create_run(
        project_name="diabetes-regression", run_name="SVR-with-QT"
    )

    run.log_params(regressor.get_params())
    run.log_metrics({"score": model.score(X_test, y_test)})
    run.log_plots({"actual_vs_predicted": plt})

    model_version = run.log_model(
        name="diabetes-regression",
        model=model,
        framework="sklearn",
        description="SVC model trained on initial data",
        model_schema={
          "features": [
            {"name": c, "type": "float"} for c in X.columns
          ],
          "prediction": "numeric",
        },
        custom_metrics=[{"name": "mean_square_error", "type": "metric", "value_type": "float"}]
    )
    print(f"Logged model: {model_version.fqn}")
    run.end()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--kernel", default="linear", type=str)
    args = parser.parse_args()

    train(kernel=args.kernel)

Overwriting train.py


In [ ]:
%%writefile requirements.txt
pandas==1.3.5
scikit-learn==1.2.1
mlfoundry==0.6.1
matplotlib==3.2.2

Overwriting requirements.txt


In [ ]:
import logging
from getpass import getpass

from servicefoundry import Build, Job, PythonBuild

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="train",
    image=Build(
        build_spec=PythonBuild(
            command="python train.py",
            requirements_path="requirements.txt",
        )
    ),
    env={
        "TFY_API_KEY": TFY_API_KEY
    }
)
deployment = job.deploy(workspace_fqn=WORKSPACE)

2023-01-31 09:06:08,629 servicefoundry INFO     Uploading code for job 'train'
2023-01-31 09:06:08,631 servicefoundry INFO     Archiving contents of dir: '/content/demo/regression'
2023-01-31 09:06:08,633 servicefoundry INFO     Neither `.tfyignore` file found in /content/demo/regression nor a valid git repository found. We recommend you to create .tfyignore file and add file patterns to ignore
Packaging source code: 1it [00:00, 437.77it/s]
2023-01-31 09:06:08,650 servicefoundry INFO     Code archive size: '1.72 KiB'
Uploading package: 100%|██████████| 1.72k/1.72k [00:00<00:00, 3.51kB/s]
2023-01-31 09:06:12,231 servicefoundry INFO     🚀 Deployment started for application 'train'. Deployment FQN is 'tfy-cluster-euwe1:kotak-demo:train:11'.
2023-01-31 09:06:13,178 servicefoundry INFO     You can track the progress below or on the dashboard:- 'https://app.truefoundry.com/applications/cldivuvwea4f41hpvffo09l9l?tab=deployments'
You can press Ctrl + C to exit the tailing of build logs and dep

Output()

2023-01-31 09:06:14,490 servicefoundry INFO     State: 'INITIALIZED'
2023-01-31 09:06:19,820 servicefoundry INFO     State: 'BUILDING'
2023-01-31 09:06:21,092 servicefoundry INFO     Tailing build logs for 'train'


Waiting for the task to start...

[2023-01-31T09:06:33.291358+00:00] [TFY-CLIENT][Start] Downloading the code

...

[2023-01-31T09:07:43.291885+00:00] time="2023-01-31T09:07:35.806Z" level=info msg="sub-process exited" argo=true 
error="<nil>"

[2023-01-31T09:07:53.293484+00:00] [TFY-CLIENT][Done] Image Built Successfully.

[2023-01-31T09:07:53.293487+00:00] [TFY-CLIENT][Done] Kubernetes deployment triggered. It may take 5-10s for the 
application to be live.

2023-01-31 09:08:01,241 servicefoundry INFO     State: 'DEPLOY_SUCCESS'


2023-01-31 09:08:02,183 servicefoundry INFO     You can find the application on the dashboard:- 'https://app.truefoundry.com/applications/cldivuvwea4f41hpvffo09l9l?tab=deployments'


In [ ]:
from servicefoundry.internal.experimental import trigger_job

JOB_DEPLOYMENT_FQN=input("Enter Job Deployment FQN:-")

for kernel in ["linear", "rbf", "poly"]:
    trigger_job(deployment_fqn=JOB_DEPLOYMENT_FQN, command=f"python train.py --kernel {kernel}")

Enter Job Deployment FQN:-tfy-cluster-euwe1:kotak-demo:train:11


2023-01-31 09:09:29,899 servicefoundry WARNING  Warning: This feature trigger_job is in experimental stage. As such there is no guarantees this will be maintained with backward compatibility or even available moving forward
2023-01-31 09:09:32,295 servicefoundry WARNING  Warning: This feature trigger_job is in experimental stage. As such there is no guarantees this will be maintained with backward compatibility or even available moving forward
2023-01-31 09:09:34,646 servicefoundry WARNING  Warning: This feature trigger_job is in experimental stage. As such there is no guarantees this will be maintained with backward compatibility or even available moving forward


# ☁ Deploy ML Model using a Service (FastAPI)
In this section we will deploy the model trained and logged by our training job as an API

In [ ]:
!mkdir -p /content/demo/regression/
%cd /content/demo/regression/

/content/demo/regression


In [ ]:
%%writefile main.py
import os
from typing import List
import uuid
from datetime import datetime

import mlfoundry as mlf
import pandas as pd
from pydantic import BaseModel
from fastapi import FastAPI

app = FastAPI(docs_url="/")


MODEL_VERSION_FQN = os.environ["MODEL_VERSION_FQN"]
client = mlf.get_client()
model = client.get_model(MODEL_VERSION_FQN).load()


class Instance(BaseModel):
    age: float
    sex: float
    bmi: float
    bp: float
    s1: float
    s2: float
    s3: float
    s4: float
    s5: float
    s6: float


class Request(BaseModel):
    instances: List[Instance]

class Response(BaseModel):
    predictions: List[float]


@app.post("/predict", response_model=Response)
def predict(request: Request):
    features = request.dict()["instances"]

    features_df = pd.DataFrame(features)
    predictions = [float(p) for p in model.predict(features_df)]
    client.log_predictions(
        model_version_fqn=MODEL_VERSION_FQN,
        predictions=[
            mlf.Prediction(
                data_id=uuid.uuid4().hex,
                features=feature,
                prediction_data={
                    "value": prediction,
                },
                occurred_at=datetime.utcnow(),
                raw_data={"data": "any_data"},
            )
            for feature, prediction in zip(features, predictions)
        ],
    )
    return Response(predictions=predictions)

Overwriting main.py


In [ ]:
%%writefile predict_requirements.txt
pandas==1.3.5
scikit-learn==1.2.1
mlfoundry==0.6.1
fastapi==0.81.0
uvicorn==0.18.3

Overwriting predict_requirements.txt


In [ ]:
MODEL_VERSION_FQN = input("Please enter the model version fqn:-")

Please enter the model version fqn:-model:truefoundry/debajyoti-tfy/diabetes-regression/diabetes-regression:8


In [ ]:
import logging
from getpass import getpass
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from servicefoundry import Build, Service, PythonBuild

service = Service(
    name="deploy",
    image=Build(
        build_spec=PythonBuild(
            command="uvicorn main:app --port 8000 --host 0.0.0.0",
            requirements_path="predict_requirements.txt",
        )
    ),
    ports=[{"port": 8000}],
    env={
        "TFY_API_KEY": TFY_API_KEY,
        "MODEL_VERSION_FQN": MODEL_VERSION_FQN
    },
)
deployment = service.deploy(workspace_fqn=WORKSPACE)

2023-01-31 09:21:37,148 servicefoundry INFO     Uploading code for service 'deploy'
2023-01-31 09:21:37,151 servicefoundry INFO     Archiving contents of dir: '/content/demo/regression'
2023-01-31 09:21:37,154 servicefoundry INFO     Neither `.tfyignore` file found in /content/demo/regression nor a valid git repository found. We recommend you to create .tfyignore file and add file patterns to ignore
Packaging source code: 1it [00:00, 353.59it/s]
2023-01-31 09:21:37,168 servicefoundry INFO     Code archive size: '1.71 KiB'
Uploading package: 100%|██████████| 1.71k/1.71k [00:00<00:00, 3.45kB/s]
2023-01-31 09:21:40,834 servicefoundry INFO     🚀 Deployment started for application 'deploy'. Deployment FQN is 'tfy-cluster-euwe1:kotak-demo:deploy:11'.
2023-01-31 09:21:41,776 servicefoundry INFO     You can track the progress below or on the dashboard:- 'https://app.truefoundry.com/applications/cldixiuxna9w91gpuf6k2djoe?tab=deployments'
You can press Ctrl + C to exit the tailing of build logs 

Output()

2023-01-31 09:21:43,043 servicefoundry INFO     State: 'INITIALIZED'
2023-01-31 09:21:53,706 servicefoundry INFO     State: 'BUILDING'
2023-01-31 09:21:54,995 servicefoundry INFO     Tailing build logs for 'deploy'


Waiting for the task to start...

[2023-01-31T09:22:01.369055+00:00] [TFY-CLIENT][Start] Downloading the code

...

[2023-01-31T09:23:21.899221+00:00] time="2023-01-31T09:23:21.899Z" level=info msg="sub-process exited" argo=true 
error="<nil>"

[2023-01-31T09:23:31.339486+00:00] [TFY-CLIENT][Done] Image Built Successfully.

[2023-01-31T09:23:31.339504+00:00] [TFY-CLIENT][Done] Kubernetes deployment triggered. It may take 5-10s for the 
application to be live.

2023-01-31 09:23:46,644 servicefoundry INFO     State: 'DEPLOY_SUCCESS'


2023-01-31 09:23:47,586 servicefoundry INFO     You can find the application on the dashboard:- 'https://app.truefoundry.com/applications/cldixiuxna9w91gpuf6k2djoe?tab=deployments'


In [ ]:
import mlfoundry

client = mlfoundry.get_client()

client.log_actuals(
    model_version_fqn=MODEL_VERSION_FQN,
    actuals=[mlfoundry.Actual(data_id="88831787f8ac4c3b80ccfc0f709bf143", value=130)],
)
